In [84]:
import pandas as pd 
import sklearn as sk
import numpy as np
import talib 
from talib import abstract
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
from xgboost import XGBRegressor, XGBClassifier
import tuneta
from sklearn.model_selection import GridSearchCV
import pandas_ta as ta
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE,RandomOverSampler, ADASYN
from sklearn.metrics import confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt



In [8]:
result = pd.read_csv('./result.csv')
margin = pd.read_csv("100_feature_past_6_months_margin_short.csv")

In [9]:
pattern = [name for name in list(result.columns) if name[:3] == 'cdl']

pattern2drop = \
    ['cdl3starsinsouth', 'cdlabandonedbaby' , 'cdlconcealbabyswall'] + \
    ['cdlbreakaway' , 'cdl2crows', 'cdl3blackcrows','cdleveningdojistar' , 'cdleveningdojistar' ] + \
    ['cdl3linestrike' , 'cdl3whitesoldiers' ]

pattern = list(set(pattern) - set(pattern2drop))


In [10]:
def classify_to_binary(profit):
    if profit > 24:
        return 0
    else:
        return 1

def classify(profit):
    if profit > 24:
        return 0
    elif profit >= 13:
        return 1
    elif profit >= 1.08:
        return 2
    else :
        return 3

In [11]:
result['date'] = pd.to_datetime(result['date'])
result['buy_date'] = pd.to_datetime(result['buy_date'])
result['sell_date'] = pd.to_datetime(result['sell_date'])
result["margin_diff"] = 0
result["short_diff"]  = 0
margin_ = margin.drop_duplicates(subset=['date','stock_id'], keep='first')
margin_["date"] = pd.to_datetime(margin_["date"])
no_margin = []

/var/folders/3c/7p2nklkn5cj5k8307c335hcc0000gn/T/ipykernel_73289/357526126.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  margin_["date"] = pd.to_datetime(margin_["date"])


In [12]:
for stock_id in result["stock_id"].unique():
    tsm = result[result["stock_id"] == stock_id]
    for i in range(len(tsm)):
        try:
            test = margin_[(margin_["stock_id"] == stock_id) & (margin_["date"] <= tsm["sell_date"].iloc[i])].sort_values(by="date", ascending=True)[-20:]
            test["margin_diff"] = test["margin_balance"].diff()
            test["short_diff"] = test["short_balance"].diff()
            if ((test["margin_diff"].iloc[-1] > 5 * test["margin_diff"].describe()["mean"]) and (test["margin_diff"].iloc[-1] == test["margin_diff"].max())) or ((test["margin_diff"].iloc[-1] == test["margin_diff"].max()) and (test["margin_diff"].describe()["mean"] < 0)):
                result.loc[result[result["stock_id"] == stock_id].index[i], 'margin_diff'] = 1
            else:
                result.loc[result[result["stock_id"] == stock_id].index[i], 'margin_diff'] = 0
            if ((test["short_diff"].iloc[-1] > 5 * test["short_diff"].describe()["mean"]) and (test["short_diff"].iloc[-1] == test["short_diff"].max())) or ((test["short_diff"].iloc[-1] == test["short_diff"].max()) and (test["short_diff"].describe()["mean"] < 0)):
                result.loc[result[result["stock_id"] == stock_id].index[i], 'short_diff'] = 1
            else:
                result.loc[result[result["stock_id"] == stock_id].index[i], 'short_diff'] = 0
        except IndexError:
            no_margin.append(stock_id)

In [13]:
test = result.copy()
test["binary classify"] = test["profit(%)"].apply(classify_to_binary)
test["non binary"] = test["profit(%)"].apply(classify)

In [14]:
for col in test.columns:
    if test[col].dtype == "int32":
        test[col] = test[col].astype("float32")
    if test[col].dtype == "float64":
        test[col] = test[col].astype("float32") 

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)


In [27]:

momentum = ['aroonup', 'aroondown', 'aroonosc', 'mom']  # momentum indicators, Josh

overlap = ['middleband' ,'midpoint' ,'midprice']  # overlap studies, Josh

volume = []  # volume indicators, Wenxuan

volatility = ['natr']

price_transform = ['wclprice']

cycle = ['ht_dcphase','date','margin_diff','short_diff']

In [34]:
keep_columns = pattern + momentum + overlap + volatility + price_transform + ['date']

# Preprocess

In [79]:
X = test.copy().drop(columns=["Unnamed: 0_x","open","close","high","low", "stock_id","acos","asin","profit(%)","buy_price","buy_date", "sell_date","binary classify","volume","amount","cosh","exp","sinh","Unnamed: 0_y","sell_price","sell_price","lowest_price","highest_price","MAE","BMFE","GMFE","non binary"])[keep_columns]

X_train = X[X['date'].dt.year <= 2020].drop(columns=["date"])
X_test = X[X['date'].dt.year >= 2021].drop(columns=["date"])
y_train = test[test['date'].dt.year <= 2020][["binary classify"]]
y_test = test[test['date'].dt.year >= 2021][["binary classify"]]

In [72]:
ros = RandomOverSampler(random_state=42,sampling_strategy='minority')

In [75]:
X_train, y_train = ros.fit_resample(X_train, y_train)


# Binary Model

In [93]:
param_grid = {
    'n_estimators': [10,50, 100, 150, 200],
    'max_depth': [1,2,3, 4, 5, 6],
    'learning_rate': [0.01, 0.05, 0.1, 0.5,0.03,0.07],
}

In [94]:
grid_search = GridSearchCV(estimator=XGBClassifier(), param_grid= param_grid, scoring='accuracy', cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1, 0.5, 0.03, 0.07],
                         'max_depth': [1, 2, 3, 4, 5, 6],
                         'n_estimators': [10, 50, 100, 150, 200]},
             scoring='accuracy')

In [96]:
best_model = grid_search.best_estimator_
test_score = best_model.score(X_test, y_test)
print("Test set score: {:.2f}".format(test_score))


Test set score: 0.96


In [76]:
xgb_binary = XGBClassifier().fit(X_train, y_train)


# Score

In [78]:
print("train_acu",accuracy_score(xgb_binary.predict(X_train),y_train))
print("test_acu",accuracy_score(xgb_binary.predict(X_test),y_test))

train_acu 1.0
test_acu 0.954331099528419


# Save model

In [97]:
import pickle
model = grid_search
with open('xgb_binary.pkl', 'wb') as file:
    pickle.dump(model, file)


# 4-Class Model

In [98]:
X = test.copy().drop(columns=["Unnamed: 0_x","open","close","high","low", "stock_id","acos","asin","profit(%)","buy_price","buy_date", "sell_date","binary classify","volume","amount","cosh","exp","sinh","Unnamed: 0_y","sell_price","sell_price","lowest_price","highest_price","MAE","BMFE","GMFE","non binary"])

X_train = X[X['date'].dt.year <= 2020].drop(columns=["date"])
X_test = X[X['date'].dt.year >= 2021].drop(columns=["date"])
y_train = test[test['date'].dt.year <= 2020][["non binary"]]
y_test = test[test['date'].dt.year >= 2021][["non binary"]]

In [99]:
param_grid = {
    'n_estimators': [10,50, 100, 150, 200],
    'max_depth': [1,2,3, 4],
    'learning_rate': [0.01, 0.1, 0.5,0.03,0.07],
}

In [106]:
X_train, y_train = ros.fit_resample(X_train, y_train)


In [107]:
grid_search = GridSearchCV(estimator=XGBClassifier(), param_grid= param_grid, scoring='accuracy', cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1, 0.5, 0.03, 0.07],
                         'max_depth': [1, 2, 3, 4, 5, 6],
                         'n_estimators': [10, 50, 100, 150, 200]},
             scoring='accuracy')

In [108]:
grid_search.best_params_

{'learning_rate': 0.5, 'max_depth': 5, 'n_estimators': 200}

In [109]:
best_model = grid_search.best_estimator_
test_score = best_model.score(X_test, y_test)
print("Test set score: {:.2f}".format(test_score))


Test set score: 0.48


In [67]:
xgb_four_class = XGBClassifier().fit(X_train, y_train)

# Score

In [68]:
print("train_acu",accuracy_score(xgb_four_class.predict(X_train),y_train))
print("test_acu",accuracy_score(xgb_four_class.predict(X_test),y_test))

train_acu 0.9986830050389373
test_acu 0.48399106478034254


# Save Model

In [82]:
import pickle
model = xgb_binary
with open('xgb_four_class.pkl', 'wb') as file:
    pickle.dump(model, file)
